# Latent Dirichlet Allocation (LDA)

In [1]:
# 親のフォルダのパスを追加
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielensのデータの読み込み
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [3]:
import gensim
import logging
from gensim.corpora.dictionary import Dictionary

movie_content = movielens.item_content.copy()
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:list(map(str, x)))

# タグとジャンルデータを使って、LDAを学習する
tag_genre_data = movie_content.tag_genre.tolist()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

# LDAの学習
lda_model = gensim.models.LdaModel(common_corpus, id2word=common_dictionary, num_topics=50, passes=30)
lda_topics = lda_model[common_corpus]




2022-12-10 20:28:31,096 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-12-10 20:28:31,343 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2022-12-10 20:28:31,366 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2022-12-10 20:28:31,370 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2022-12-10T20:28:31.368643', 'gensim': '4.2.0', 'python': '3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2022-12-10 20:28:31,538 : INFO : using symmetric alpha at 0.02
2022-12-10 20:28:31,539 : INFO : using symmetric eta at 0.02
2022-12-10 20:28:31,543 : INFO : us

2022-12-10 20:28:35,317 : INFO : topic #2 (0.020): 0.026*"martin scorsese" + 0.025*"surreal" + 0.016*"Comedy" + 0.015*"based on a comic" + 0.015*"los angeles" + 0.015*"wedding" + 0.014*"existentialism" + 0.014*"nudity (rear)" + 0.013*"visually appealing" + 0.013*"cult film"
2022-12-10 20:28:35,318 : INFO : topic #13 (0.020): 0.052*"drugs" + 0.034*"ummarti2006" + 0.029*"Drama" + 0.019*"dani2006" + 0.019*"sex" + 0.017*"soccer" + 0.017*"ewan mcgregor" + 0.016*"male nudity" + 0.015*"intimate" + 0.014*"gritty"
2022-12-10 20:28:35,321 : INFO : topic diff=0.235841, rho=0.447214
2022-12-10 20:28:35,554 : INFO : -21.344 per-word bound, 2662525.3 perplexity estimate based on a held-out corpus of 681 documents with 5199 words
2022-12-10 20:28:35,555 : INFO : PROGRESS: pass 0, at document #10681/10681
2022-12-10 20:28:35,688 : INFO : merging changes from 681 documents into a model of 10681 documents
2022-12-10 20:28:35,725 : INFO : topic #4 (0.020): 0.051*"espionage" + 0.049*"edward norton" + 0.03

2022-12-10 20:28:37,300 : INFO : topic diff=0.194125, rho=0.369094
2022-12-10 20:28:37,302 : INFO : PROGRESS: pass 1, at document #10000/10681
2022-12-10 20:28:37,633 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:28:37,666 : INFO : topic #26 (0.020): 0.203*"r" + 0.112*"movie to see" + 0.099*"clearplay" + 0.057*"Drama" + 0.039*"easily confused with other movie(s) (title)" + 0.035*"Action" + 0.031*"revenge" + 0.029*"Thriller" + 0.025*"Comedy" + 0.021*"video game adaptation"
2022-12-10 20:28:37,668 : INFO : topic #43 (0.020): 0.040*"Comedy" + 0.037*"dark comedy" + 0.031*"quirky" + 0.028*"black comedy" + 0.026*"baseball" + 0.023*"coen brothers" + 0.022*"oppl" + 0.021*"seen 2008" + 0.019*"light" + 0.019*"joaquin phoenix"
2022-12-10 20:28:37,669 : INFO : topic #18 (0.020): 0.197*"War" + 0.162*"Drama" + 0.049*"nudity (full frontal)" + 0.043*"Action" + 0.039*"not corv lib" + 0.031*"directorial debut" + 0.026*"george clooney" + 0.026*"hw drama" + 0.0

2022-12-10 20:28:41,908 : INFO : topic #49 (0.020): 0.089*"twist ending" + 0.058*"nicolas cage" + 0.055*"philip k. dick" + 0.044*"kevin spacey" + 0.042*"sexuality" + 0.024*"monster" + 0.021*"1970s" + 0.018*"poker" + 0.018*"jodie foster" + 0.013*"circus"
2022-12-10 20:28:41,910 : INFO : topic #27 (0.020): 0.096*"james bond" + 0.066*"007" + 0.061*"bond" + 0.025*"tolkien" + 0.024*"murder" + 0.021*"franchise" + 0.020*"Adventure" + 0.018*"Action" + 0.018*"ireland" + 0.016*"Thriller"
2022-12-10 20:28:41,911 : INFO : topic #37 (0.020): 0.254*"based on a book" + 0.129*"adapted from:book" + 0.050*"magic" + 0.037*"based on book" + 0.028*"Drama" + 0.024*"surrealism" + 0.022*"classic" + 0.014*"kate" + 0.011*"private detective" + 0.011*"dvd"
2022-12-10 20:28:41,913 : INFO : topic #5 (0.020): 0.111*"fantasy" + 0.056*"sci-fi" + 0.048*"adventure" + 0.046*"space" + 0.029*"dvd" + 0.029*"seen more than once" + 0.022*"sequel" + 0.018*"seen at the cinema" + 0.015*"college" + 0.014*"franchise"
2022-12-10 20

2022-12-10 20:28:44,921 : INFO : topic #49 (0.020): 0.097*"twist ending" + 0.060*"nicolas cage" + 0.060*"philip k. dick" + 0.046*"kevin spacey" + 0.043*"sexuality" + 0.025*"1970s" + 0.025*"monster" + 0.024*"jodie foster" + 0.017*"poker" + 0.014*"twist"
2022-12-10 20:28:44,922 : INFO : topic #48 (0.020): 0.069*"Drama" + 0.046*"criterion" + 0.028*"dvd-video" + 0.026*"tumey's dvds" + 0.025*"reflective" + 0.022*"lyrical" + 0.022*"deliberate" + 0.020*"poignant" + 0.020*"quirky" + 0.019*"bittersweet"
2022-12-10 20:28:44,925 : INFO : topic diff=0.115431, rho=0.327201
2022-12-10 20:28:44,928 : INFO : PROGRESS: pass 3, at document #8000/10681
2022-12-10 20:28:45,262 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:28:45,298 : INFO : topic #37 (0.020): 0.267*"based on a book" + 0.136*"adapted from:book" + 0.061*"magic" + 0.038*"based on book" + 0.030*"Drama" + 0.025*"surrealism" + 0.014*"kate" + 0.013*"classic" + 0.012*"dvd" + 0.011*"private detective"
2

...

2022-12-10 20:29:54,040 : INFO : topic #7 (0.020): 0.130*"dystopia" + 0.060*"Sci-Fi" + 0.057*"pg-13" + 0.053*"robots" + 0.050*"futuristmovies.com" + 0.048*"remade" + 0.039*"post apocalyptic" + 0.017*"2" + 0.014*"space travel" + 0.013*"friends should see"
2022-12-10 20:29:54,042 : INFO : topic #25 (0.020): 0.314*"Documentary" + 0.192*"betamax" + 0.064*"in netflix queue" + 0.023*"IMAX" + 0.021*"notable nudity" + 0.018*"dvd-video" + 0.016*"crappy sequel" + 0.015*"robert downey jr" + 0.012*"no dialogue" + 0.010*"short"
2022-12-10 20:29:54,042 : INFO : topic #44 (0.020): 0.166*"oscar (best picture)" + 0.066*"oscar (best directing)" + 0.064*"oscar (best actor)" + 0.052*"oscar (best actress)" + 0.047*"oscar (best supporting actress)" + 0.047*"ghosts" + 0.026*"peter jackson" + 0.026*"england" + 0.025*"overrated" + 0.025*"mental illness"
2022-12-10 20:29:54,043 : INFO : topic #29 (0.020): 0.149*"anime" + 0.128*"remake" + 0.068*"japan" + 0.047*"sean connery" + 0.045*"cars" + 0.030*"jane austen" 

2022-12-10 20:29:56,049 : INFO : topic #22 (0.020): 0.060*"sci-fi" + 0.058*"jim carrey" + 0.047*"funny" + 0.039*"will smith" + 0.031*"tommy lee jones" + 0.028*"virtual reality" + 0.028*"stupid" + 0.027*"memory" + 0.027*"comedy" + 0.024*"get"
2022-12-10 20:29:56,050 : INFO : topic #45 (0.020): 0.045*"atmospheric" + 0.041*"tense" + 0.041*"disturbing" + 0.032*"zombies" + 0.031*"stylized" + 0.019*"visceral" + 0.019*"menacing" + 0.018*"ominous" + 0.014*"bleak" + 0.014*"erlend's dvds"
2022-12-10 20:29:56,050 : INFO : topic #36 (0.020): 0.132*"true story" + 0.114*"based on a true story" + 0.036*"dustin hoffman" + 0.025*"australian" + 0.018*"bank robbery" + 0.015*"tumey's vhs" + 0.015*"emotion!" + 0.013*"ron howard" + 0.013*"william shatner" + 0.012*"russia"
2022-12-10 20:29:56,053 : INFO : topic diff=0.022301, rho=0.173186
2022-12-10 20:29:56,055 : INFO : PROGRESS: pass 27, at document #6000/10681
2022-12-10 20:29:56,352 : INFO : merging changes from 2000 documents into a model of 10681 docum

2022-12-10 20:29:58,259 : INFO : topic #43 (0.020): 0.075*"quirky" + 0.074*"Comedy" + 0.049*"dark comedy" + 0.042*"humorous" + 0.042*"coen brothers" + 0.033*"oppl" + 0.033*"irreverent" + 0.031*"black comedy" + 0.025*"witty" + 0.025*"satirical"
2022-12-10 20:29:58,262 : INFO : topic diff=0.128268, rho=0.170646
2022-12-10 20:29:58,264 : INFO : PROGRESS: pass 28, at document #4000/10681
2022-12-10 20:29:58,544 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:29:58,579 : INFO : topic #9 (0.020): 0.050*"pirates" + 0.043*"sean penn" + 0.031*"infidelity" + 0.028*"dogs" + 0.028*"marx brothers" + 0.023*"blaxploitation" + 0.022*"italy" + 0.020*"epic" + 0.019*"brian de palma" + 0.018*"audrey hepburn"
2022-12-10 20:29:58,580 : INFO : topic #26 (0.020): 0.243*"r" + 0.124*"clearplay" + 0.123*"movie to see" + 0.065*"Drama" + 0.034*"easily confused with other movie(s) (title)" + 0.027*"to see" + 0.025*"russell crowe" + 0.022*"video game adaptation" + 0.021*"ph

2022-12-10 20:30:00,008 : INFO : topic diff=0.068993, rho=0.170646
2022-12-10 20:30:00,010 : INFO : PROGRESS: pass 29, at document #2000/10681
2022-12-10 20:30:00,349 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:30:00,386 : INFO : topic #39 (0.020): 0.091*"politics" + 0.080*"Drama" + 0.075*"biography" + 0.061*"music" + 0.049*"library" + 0.037*"biopic" + 0.029*"samurai" + 0.028*"akira kurosawa" + 0.023*"japan" + 0.019*"history"
2022-12-10 20:30:00,387 : INFO : topic #33 (0.020): 0.060*"vampire" + 0.047*"vampires" + 0.045*"gothic" + 0.039*"christianity" + 0.038*"Horror" + 0.035*"religion" + 0.031*"erlend's dvds" + 0.029*"netwatch" + 0.028*"scary movies to see on halloween" + 0.023*"m. night shyamalan"
2022-12-10 20:30:00,388 : INFO : topic #5 (0.020): 0.081*"sci-fi" + 0.077*"fantasy" + 0.060*"space" + 0.043*"adventure" + 0.031*"dvd" + 0.026*"harrison ford" + 0.026*"seen more than once" + 0.022*"action" + 0.022*"sequel" + 0.014*"Adventure"
202

2022-12-10 20:30:02,097 : INFO : topic #8 (0.020): 0.091*"tim burton" + 0.085*"family" + 0.044*"watched 2007" + 0.038*"dark" + 0.023*"sandra bullock" + 0.020*"halloween" + 0.015*"twins" + 0.014*"letters" + 0.014*"dramatic" + 0.014*"foul language"
2022-12-10 20:30:02,098 : INFO : topic #21 (0.020): 0.339*"nudity (topless)" + 0.207*"nudity (topless - brief)" + 0.110*"johnny depp" + 0.043*"Drama" + 0.034*"jack nicholson" + 0.012*"to see" + 0.011*"united states" + 0.008*"biographical view" + 0.007*"adapted from:book series" + 0.007*"benicio del toro"
2022-12-10 20:30:02,100 : INFO : topic #13 (0.020): 0.107*"drugs" + 0.032*"gritty" + 0.030*"ummarti2006" + 0.020*"intimate" + 0.019*"male nudity" + 0.017*"ewan mcgregor" + 0.016*"addiction" + 0.016*"good dialogue" + 0.016*"kevin smith" + 0.015*"sex"
2022-12-10 20:30:02,100 : INFO : topic #28 (0.020): 0.085*"disney" + 0.076*"animation" + 0.069*"pixar" + 0.038*"Animation" + 0.032*"children" + 0.029*"Children" + 0.026*"fairy tale" + 0.025*"pg" + 

In [4]:
# LDAContentレコメンド
from src.lda_content import LDAContentRecommender
recommender = LDAContentRecommender()
recommend_result = recommender.recommend(movielens)

2022-12-10 20:30:02,815 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-12-10 20:30:02,954 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2022-12-10 20:30:02,962 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2022-12-10 20:30:02,962 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2022-12-10T20:30:02.962905', 'gensim': '4.2.0', 'python': '3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2022-12-10 20:30:03,193 : INFO : using symmetric alpha at 0.02
2022-12-10 20:30:03,193 : INFO : using symmetric eta at 0.02
2022-12-10 20:30:03,196 : INFO : us

2022-12-10 20:30:05,498 : INFO : topic #16 (0.020): 0.080*"nudity (rear)" + 0.053*"pg" + 0.041*"military" + 0.035*"claymation" + 0.031*"weird" + 0.030*"nudity (topless)" + 0.024*"aardman" + 0.019*"nudity (topless - brief)" + 0.019*"childhood" + 0.018*"penguins"
2022-12-10 20:30:05,501 : INFO : topic diff=0.241935, rho=0.447214
2022-12-10 20:30:05,700 : INFO : -21.496 per-word bound, 2957512.5 perplexity estimate based on a held-out corpus of 681 documents with 5199 words
2022-12-10 20:30:05,702 : INFO : PROGRESS: pass 0, at document #10681/10681
2022-12-10 20:30:05,815 : INFO : merging changes from 681 documents into a model of 10681 documents
2022-12-10 20:30:05,849 : INFO : topic #36 (0.020): 0.035*"russell crowe" + 0.031*"virus" + 0.029*"not available from netflix" + 0.028*"Comedy" + 0.026*"based on a book" + 0.025*"cult film" + 0.025*"peter jackson" + 0.024*"movielens quickpick" + 0.023*"1980s" + 0.021*"adam sandler"
2022-12-10 20:30:05,850 : INFO : topic #6 (0.020): 0.125*"Musical

2022-12-10 20:30:07,549 : INFO : topic #42 (0.020): 0.086*"martial arts" + 0.059*"robin williams" + 0.042*"jonossa" + 0.041*"seen 2006" + 0.031*"japan" + 0.029*"samurai" + 0.027*"Drama" + 0.025*"orson welles" + 0.024*"hugh grant" + 0.023*"divorce"
2022-12-10 20:30:07,551 : INFO : topic #41 (0.020): 0.141*"remake" + 0.058*"new york city" + 0.049*"christmas" + 0.044*"family" + 0.037*"new york" + 0.031*"Drama" + 0.027*"seen" + 0.024*"kevin spacey" + 0.023*"predictable" + 0.021*"friday night movie"
2022-12-10 20:30:07,552 : INFO : topic #5 (0.020): 0.162*"classic" + 0.082*"musical" + 0.027*"afi 100 (laughs)" + 0.024*"los angeles" + 0.018*"witch" + 0.018*"national film registry" + 0.016*"john travolta" + 0.015*"Musical" + 0.014*"adapted from b'way" + 0.013*"movie to see"
2022-12-10 20:30:07,553 : INFO : topic #49 (0.020): 0.081*"national film registry" + 0.026*"natalie portman" + 0.025*"tumey's dvds" + 0.022*"Film-Noir" + 0.020*"film noir" + 0.020*"black and white" + 0.019*"imdb top 250" + 

2022-12-10 20:30:09,204 : INFO : topic #45 (0.020): 0.235*"less than 300 ratings" + 0.109*"Drama" + 0.061*"nudity (topless - notable)" + 0.036*"Comedy" + 0.035*"tim burton" + 0.023*"library" + 0.018*"michael moore" + 0.015*"blindfold" + 0.014*"depressing" + 0.011*"golden raspberry (worst actor)"
2022-12-10 20:30:09,205 : INFO : topic #37 (0.020): 0.185*"War" + 0.109*"world war ii" + 0.103*"Drama" + 0.047*"history" + 0.047*"war" + 0.035*"jim carrey" + 0.025*"Action" + 0.019*"70mm" + 0.014*"nazis" + 0.013*"mental illness"
2022-12-10 20:30:09,206 : INFO : topic #38 (0.020): 0.142*"drugs" + 0.037*"samuel l. jackson" + 0.035*"peter sellers" + 0.030*"ewan mcgregor" + 0.024*"divx" + 0.022*"poverty" + 0.020*"addiction" + 0.019*"hulu" + 0.018*"michael caine" + 0.017*"Drama"
2022-12-10 20:30:09,208 : INFO : topic diff=0.119826, rho=0.346261
2022-12-10 20:30:09,209 : INFO : PROGRESS: pass 2, at document #10000/10681
2022-12-10 20:30:09,510 : INFO : merging changes from 2000 documents into a model

2022-12-10 20:30:10,861 : INFO : topic #11 (0.020): 0.127*"religion" + 0.115*"keanu reeves" + 0.020*"dark" + 0.018*"reality tv" + 0.018*"courtesan" + 0.018*"slash" + 0.017*"slavery" + 0.017*"facebook rec" + 0.014*"class issues" + 0.012*"product placement"
2022-12-10 20:30:10,863 : INFO : topic diff=0.113857, rho=0.327201
2022-12-10 20:30:10,865 : INFO : PROGRESS: pass 3, at document #8000/10681
2022-12-10 20:30:11,147 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:30:11,182 : INFO : topic #20 (0.020): 0.086*"biography" + 0.060*"Drama" + 0.043*"biopic" + 0.042*"ingmar bergman" + 0.035*"ensemble cast" + 0.031*"julia roberts" + 0.030*"suicide" + 0.028*"corvallis library" + 0.027*"multiple storylines" + 0.025*"medieval"
2022-12-10 20:30:11,183 : INFO : topic #47 (0.020): 0.059*"serial killer" + 0.046*"psychology" + 0.045*"brad pitt" + 0.041*"edward norton" + 0.033*"heist" + 0.031*"hayao miyazaki" + 0.031*"steven spielberg" + 0.029*"crime" + 0.029

...

2022-12-10 20:31:04,875 : INFO : topic #18 (0.020): 0.082*"magic" + 0.079*"gay" + 0.062*"racism" + 0.044*"pg13" + 0.040*"Drama" + 0.027*"food" + 0.027*"social commentary" + 0.023*"homosexuality" + 0.020*"19th century" + 0.019*"denzel washington"
2022-12-10 20:31:04,876 : INFO : topic #19 (0.020): 0.062*"nudity (full frontal)" + 0.061*"drama" + 0.035*"Drama" + 0.034*"philip k. dick" + 0.028*"vietnam war" + 0.027*"rape" + 0.025*"vietnam" + 0.022*"very good" + 0.020*"to see" + 0.016*"slow"
2022-12-10 20:31:04,879 : INFO : topic diff=0.046241, rho=0.175844
2022-12-10 20:31:04,882 : INFO : PROGRESS: pass 26, at document #8000/10681
2022-12-10 20:31:05,164 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:31:05,205 : INFO : topic #2 (0.020): 0.287*"Documentary" + 0.104*"to see" + 0.064*"documentary" + 0.026*"mockumentary" + 0.025*"sexuality" + 0.023*"star trek" + 0.021*"dogs" + 0.021*"kevin smith" + 0.018*"movie to see" + 0.017*"in netflix queue"
2022

2022-12-10 20:31:06,644 : INFO : topic diff=0.023122, rho=0.173186
2022-12-10 20:31:06,650 : INFO : PROGRESS: pass 27, at document #6000/10681
2022-12-10 20:31:07,001 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:31:07,037 : INFO : topic #18 (0.020): 0.081*"magic" + 0.079*"gay" + 0.062*"racism" + 0.044*"pg13" + 0.040*"Drama" + 0.027*"food" + 0.027*"social commentary" + 0.023*"homosexuality" + 0.020*"19th century" + 0.019*"denzel washington"
2022-12-10 20:31:07,038 : INFO : topic #32 (0.020): 0.135*"johnny depp" + 0.067*"clint eastwood" + 0.059*"vhs" + 0.045*"jackie chan" + 0.037*"western" + 0.031*"kung fu" + 0.031*"leonardo dicaprio" + 0.030*"spaghetti western" + 0.029*"propaganda" + 0.029*"david lynch"
2022-12-10 20:31:07,039 : INFO : topic #26 (0.020): 0.042*"Drama" + 0.035*"james bond" + 0.024*"007" + 0.023*"bond" + 0.021*"mel gibson" + 0.021*"reflective" + 0.020*"atmospheric" + 0.019*"poignant" + 0.018*"lyrical" + 0.017*"bittersweet"
202

2022-12-10 20:31:09,001 : INFO : topic #7 (0.020): 0.113*"romance" + 0.065*"Romance" + 0.054*"chick flick" + 0.038*"boring" + 0.030*"girlie movie" + 0.027*"baseball" + 0.023*"Comedy" + 0.020*"love story" + 0.019*"whimsical" + 0.017*"wedding"
2022-12-10 20:31:09,002 : INFO : topic #5 (0.020): 0.292*"classic" + 0.062*"musical" + 0.045*"afi 100 (laughs)" + 0.034*"Musical" + 0.030*"national film registry" + 0.021*"afi 100" + 0.016*"70mm" + 0.015*"adapted from b'way" + 0.015*"john travolta" + 0.015*"breakthroughs"
2022-12-10 20:31:09,003 : INFO : topic #31 (0.020): 0.127*"zombies" + 0.074*"horror" + 0.072*"pg-13" + 0.039*"cult classic" + 0.025*"infidelity" + 0.025*"campy" + 0.024*"zombie" + 0.024*"joaquin phoenix" + 0.024*"sam raimi" + 0.020*"books"
2022-12-10 20:31:09,006 : INFO : topic diff=0.022572, rho=0.170646
2022-12-10 20:31:09,008 : INFO : PROGRESS: pass 28, at document #6000/10681
2022-12-10 20:31:09,254 : INFO : merging changes from 2000 documents into a model of 10681 documents
2

2022-12-10 20:31:10,473 : INFO : topic #17 (0.020): 0.330*"Sci-Fi" + 0.071*"Horror" + 0.068*"Action" + 0.044*"video game adaptation" + 0.027*"animals" + 0.025*"movie to see" + 0.022*"football" + 0.018*"futuristic" + 0.015*"milla jovovich" + 0.013*"g"
2022-12-10 20:31:10,474 : INFO : topic #34 (0.020): 0.074*"ghosts" + 0.051*"bill murray" + 0.044*"television" + 0.030*"courtroom" + 0.030*"courtroom drama" + 0.026*"secret service" + 0.021*"roman polanski" + 0.017*"18th century" + 0.017*"opera" + 0.017*"court"
2022-12-10 20:31:10,476 : INFO : topic diff=0.124178, rho=0.168215
2022-12-10 20:31:10,477 : INFO : PROGRESS: pass 29, at document #4000/10681
2022-12-10 20:31:10,741 : INFO : merging changes from 2000 documents into a model of 10681 documents
2022-12-10 20:31:10,773 : INFO : topic #31 (0.020): 0.127*"zombies" + 0.074*"horror" + 0.072*"pg-13" + 0.039*"cult classic" + 0.025*"infidelity" + 0.025*"campy" + 0.024*"zombie" + 0.024*"joaquin phoenix" + 0.024*"sam raimi" + 0.020*"books"
2022

2022-12-10 20:31:11,899 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel<num_terms=15261, num_topics=50, decay=0.5, chunksize=2000> in 68.59s', 'datetime': '2022-12-10T20:31:11.899611', 'gensim': '4.2.0', 'python': '3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}


In [5]:
#  評価
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.004, Recall@K=0.012
